In [1]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import numpy as np
from utils.plot_data import plot_sensor_measurement, plot_wind_rose, plot_co2_rose


start_date = datetime(2024, 4, 7, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 4, 9, 23, 59, 59).replace(tzinfo=timezone.utc)

filter = '1h'

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

# processed 10min average measurement data
df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "10m_cal_corr_acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()


In [5]:
# raw data
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()

# Plot other data directly from local database measurements.parquet

In [2]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_humidity",filter=filter)

In [3]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_temperature",filter=filter)

In [4]:
sbs = [2,4,7,9,10,11,14,15,17,19]
plot_sensor_measurement(df,sensor_id=sbs,col_name="gmp343_corrected",filter=filter)

In [8]:
plot_sensor_measurement(df_raw,sensor_id=sensor_id,col_name="wxt532_speed_avg",filter=filter)

NameError: name 'df_raw' is not defined

In [7]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_pressure",filter=filter)

In [11]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_temperature",filter=filter)

NameError: name 'df_raw' is not defined

In [9]:
plot_sensor_measurement(df,sensor_id=sensor_id,col_name="gmp343_raw",filter=filter)

In [11]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_filtered",filter=filter)

In [14]:
plot_sensor_measurement(df_raw,sensor_id=[1,2,4,5,6,7,8,14,15],col_name="wxt532_temperature",filter=filter)

In [6]:
# plot wind at 3m pole inlet
plot_co2_rose(df,df_raw,6, 'TUM')
# side by side
plot_co2_rose(df,df_raw,5, 'TUM Inlets') # was 16 until 06.02.2024
# Maisach
plot_co2_rose(df,df_raw,16, 'Maisach') # was 1 until 06.02.2024

# Rechts-der-Isar
plot_co2_rose(df,df_raw,1, 'Klenze Gymnasium') # from 27.02.2024

# Rechts-der-Isar
plot_co2_rose(df,df_raw,8, 'Rechts-der-Isar') # was 2 until 12.02.2024
# Finsing
plot_co2_rose(df,df_raw,11, 'Finsing') # was 15 until 20.02.2024, # was 3 until 09.04.2024
# Großhadern
plot_co2_rose(df,df_raw,20, 'Großhadern') # was 4 until 12.02.2024
# Großhadern
plot_co2_rose(df,df_raw,18, 'Pasing') # was 5 until 06.02.2024
# Feldkirchen
plot_co2_rose(df,df_raw,13, 'Feldkirchen') # was 7 until 20.02.2024
# Taufkirchen
plot_co2_rose(df,df_raw,12, 'Taufkirchen') # was 8 until 12.02.2024
# Finsing
plot_co2_rose(df,df_raw,5, 'DLR') # was 14 until 26.02.2024

# Schwabing
plot_co2_rose(df,df_raw, 10, 'Schwabing') # was 14 until 26.02.2024

In [28]:
# plot wind at 3m pole inlet
plot_wind_rose(df_raw,6, 'TUM')
# side by side
plot_wind_rose(df_raw,5, 'TUM Inlets') # was 16 until 06.02.2024
# Maisach
plot_wind_rose(df_raw,16, 'Maisach') # was 1 until 06.02.2024
# Rechts-der-Isar
plot_wind_rose(df_raw,1, 'Klenze Gymnasium') # from 27.02.2024
# Rechts-der-Isar
plot_wind_rose(df_raw,8, 'Rechts-der-Isar') # was 2 until 12.02.2024
# Großhadern
plot_wind_rose(df_raw,20, 'Großhadern') # was 4 until 12.02.2024
# Großhadern
plot_wind_rose(df_raw,18, 'Pasing') # was 5 until 06.02.2024
# Feldkirchen
plot_wind_rose(df_raw,13, 'Feldkirchen') # was 7 until 20.02.2024
# Taufkirchen
plot_wind_rose(df_raw,12, 'Taufkirchen') # was 8 until 12.02.2024
# Finsing
plot_wind_rose(df_raw,11, 'DLR') # was 14 until 26.02.2024
# Schwabing
plot_co2_rose(df,df_raw, 10, 'Schwabing') # was 14 until 26.02.2024

In [17]:
plot_sensor_measurement(df_raw,sensor_id,col_name="wxt532_direction_avg",filter=filter)

In [18]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_humidity",filter=filter)

In [19]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_temperature",filter=filter)

In [20]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_temperature",filter=filter)

In [22]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_cpu_usage",filter=filter)

In [23]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_memory_usage",filter=filter)


In [24]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_disk_usage",filter=filter)


In [25]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_cpu_temperature",filter=filter)

In [26]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_humidity",filter=filter)

In [27]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_pressure",filter=filter)

# Calibration Data

In [13]:
sensor_id = [4,17,19]

In [14]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bottle_id",filter=None)

In [15]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None)

In [21]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_sht45_humidity",filter=None)

In [31]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_sht45_temperature",filter=None)

In [20]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_temperature",filter=None)

In [19]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_temperature",filter=None)

In [18]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_humidity",filter=None)

In [17]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_pressure",filter=None)

In [36]:
# Plot 400ppm Calibration Bottle 
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None,cut_above= 450,cut_below=350)

In [37]:
# Plot 800ppm Calibration Bottle 
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None,cut_below=460)